In [29]:
import pandas as pd
import sklearn.feature_extraction as fe
from sklearn.metrics.pairwise import cosine_similarity

In [23]:
df = pd.read_csv("data/bulbapedia_smogon_data.csv").set_index("name")
df.head()

,ndex,thumbnail,description
name,,,
Bulbasaur,1,https://archives.bulbagarden.net/media/upload/...,weak weak frail frail slow light short Chlorop...
Ivysaur,2,https://archives.bulbagarden.net/media/upload/...,weak weak frail frail slow light short Chlorop...
Venusaur,3,https://archives.bulbagarden.net/media/upload/...,weak weak frail frail slow light short Chlorop...
Charmander,4,https://archives.bulbagarden.net/media/upload/...,weak weak frail frail slow light short Blaze S...
Charmeleon,5,https://archives.bulbagarden.net/media/upload/...,weak weak frail frail slow light short Blaze S...


In [13]:
model = fe.text.TfidfVectorizer(input="content")
vector = model.fit_transform(df["description"])

def ir_tf_idf(query):
    res = model.transform([query])
    similarities = cosine_similarity(vector,res)
    df.assign(sim=similarities.reshape(-1)).sort_values(by="sim",ascending=False)
    return df[:5]

In [30]:
# res = model.transform([query])
# similarities = cosine_similarity(vector,res)

In [45]:
# matches = df.assign(sim=similarities.reshape(-1)).sort_values(by="sim",ascending=False)[:5]
# matches.index

Index(['Magmortar', 'Cyndaquil', 'Quilava', 'Volcanion', 'Flareon'], dtype='object', name='name')